In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import shutil
import os
import csv

In [9]:
# 請分別輸入病人csv檔案,時間間隔,開始時間
# 檔案名稱請附上結尾.csv
# 時間間隔單位為分鐘
# 開始時間預定為早上8.
# ex : visualization('01A5.csv', 30)

def visualization(file, time_step, start=8):
    df = pd.read_csv(file, header = None)
    df[0] = [dt.datetime.strptime(i, '%Y/%m/%d').date() for i in df[0].values]
    df[1] = [dt.datetime.strptime(i, '%H:%M:%S').time() for i in df[1].values]
    date_list = np.sort(list(dict.fromkeys(df[0].values)))
    
    today_date = date_list[-1] # 定義今天時間
    time_zone = file[:4] + '_' + str(today_date) + ' box plot' 
    
    date = df[0].values
    d_filter = (date>=today_date)&(date<today_date+datetime.timedelta(days = 1))
    
    data = (df[3].values)[d_filter]/100 # each date
    time = (df[1].values)[d_filter] # each hour
   
    start_datetime = datetime.datetime.combine(today_date, datetime.time(start,0))
    end_datetime = datetime.datetime.combine(today_date, time[-1])
    datetime_list = [(start_datetime+i*datetime.timedelta(minutes = time_step)) for i in range(int(np.floor((end_datetime-start_datetime)/datetime.timedelta(minutes = time_step))))]
    
    each_hour = []
    for t_idx in range(len(datetime_list)):
        t_filter = (time>=datetime_list[t_idx].time()) & (time<(datetime_list[t_idx]+datetime.timedelta(minutes = time_step)).time())
        tmp = data[(t_filter)&(data>25)&(data<45)]
        each_hour.append(tmp)

    sick_hour, sick_tmp = read_tmp_to_list('01A5_ear.csv', today_date, datetime.time(start, 0), time_step, 34)
    
    plt.figure(figsize = (10,5))
    plt.boxplot(each_hour, flierprops  = dict(markerfacecolor = 'b', marker = '.'))
    baseline = plt.plot(np.linspace(1, time_step, time_step), [35 for i in range(time_step)], 'g:')
    sick_points = plt.scatter(sick_hour, sick_tmp,  marker = 'D', color='r' )
    
    l1 = plt.legend(baseline, ['baseline'], loc='lower right')
    plt.legend([sick_points], ['sick points'], loc='upper right')
    plt.gca().add_artist(l1)

    time_tag = [datetime.datetime.strftime(i, "%H:%M") for i in datetime_list]
    plt.xticks(np.linspace(1, time_step, time_step), time_tag, rotation='vertical')
    plt.xlabel('Hours')
    plt.ylabel('Temperature')
    plt.suptitle(time_zone)
#     plt.savefig( time_zone + '.png', dpi = 300)
    plt.show()


In [10]:
# 讀取耳溫量測表的資料 在主程式會用到

def read_tmp_to_list(file, today_date, today_start_time, time_step, warning_cri = 37):
    
    df = pd.read_csv(file, header = None, skiprows=1)
    df[0] = [dt.datetime.strptime(i, '%Y-%m-%d').date() for i in df[0].values]
    df[1] = [dt.datetime.strptime(i, '%H:%M').time() for i in df[1].values]
    if (type(today_date ) != datetime.date) | (type(today_start_time) != datetime.time):
        print("ERROR DATA TYPE")
        return 0
    else:
        today_datetime = datetime.datetime.combine(today_date, today_start_time)

    df_ear_warn = df[(df[2]>warning_cri)&(df[0]==today_date)&(df[1]>=today_start_time)]
    
    df_ear_warn[1] = [np.floor((datetime.datetime.combine(today_date, i)-today_datetime)/datetime.timedelta(minutes = time_step)) for i in df_ear_warn[1].values]
    
    return 1+df_ear_warn[1].values, df_ear_warn[2].values

In [11]:
# 範例執行方式 :

visualization( '01A5.csv', 30 )

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/math/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-b654dce33541>", line 3, in <module>
    visualization( '01A5.csv', 30 )
  File "<ipython-input-9-5134fd6cb6dd>", line 8, in visualization
    df = pd.read_csv(file, header = None)
  File "/home/math/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 702, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/math/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 429, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/math/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/home/math/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1122, in _make_engine
    self._engine = CParserWrapper(self.f, **se

FileNotFoundError: [Errno 2] File b'01A5.csv' does not exist: b'01A5.csv'